# Joining and Appending DataFrames in PySpark HW

Now it's time to test your knowledge and further engrain the concepts we touched on in the lectures. Let's go ahead and get started.




**As always let's start our Spark instance.**

In [1]:
# import findspark
# findspark.init()

import pyspark  # only run after findspark.init()
from pyspark.sql import SparkSession

# May take awhile locally
spark = SparkSession.builder.appName("joins").getOrCreate()

cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()
print("You are working with", cores, "core(s)")
spark

22/10/09 16:30:32 WARN Utils: Your hostname, masoud-ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.7.139 instead (on interface wlp2s0)
22/10/09 16:30:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/09 16:30:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
You are working with 1 core(s)


## Read in the database

Let continue working with our college courses dataframe to get some more insights and practice what we have learned!Let's read in the whole database using the loop function that we learned about in the lecture to automatically read in all the datasets from the uw-madision-courses folder (there are too many datasets to each one individually.

In [2]:
import os

path = "Datasets/uw-madison-courses/"

df_list = []
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        filename_list = filename.split(".")  # separate path from .csv
        df_name = filename_list[0]
        df = spark.read.csv(path + filename, inferSchema=True, header=True)
        df.name = df_name
        df_list.append(df_name)
        exec(df_name + " = df")

# QA
print("Full list of dfs:")
print(df_list)

Full list of dfs:
['courses', 'course_offerings', 'grade_distributions', 'instructors', 'rooms', 'schedules', 'sections', 'subjects', 'subject_memberships', 'teachings']


Now check the contents of a few of the dataframses that were read in above.

In [3]:
rooms.limit(3).toPandas()

,uuid,facility_code,room_code
0,04368a56-c959-3e4b-8b3d-f4cc3538fea5,OFF CAMPUS,null
1,2cc50da3-ef0e-3572-a557-ca44930a0688,0032,0249
2,ebbf62b4-2ac3-356b-b0fa-7897f4446a17,0032,B101


In [4]:
sections.limit(3).toPandas()

,uuid,course_offering_uuid,section_type,number,room_uuid,schedule_uuid
0,45adf63c-48c9-3659-8561-07556d2d4ddf,344b3ebe-da7e-314c-83ed-9425269695fd,FLD,1,null,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
1,c6280e23-5e43-3859-893e-540d94993529,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,FLD,1,null,f2d66a4d-0c08-3b48-abf6-649fffd7ae90
2,9395dc21-15d1-3fab-8d1f-6f3fe6114c48,ea3b717c-d66b-30dc-8b37-964d9688295f,FLD,1,04368a56-c959-3e4b-8b3d-f4cc3538fea5,f2d66a4d-0c08-3b48-abf6-649fffd7ae90


In [5]:
courses.limit(3).toPandas()

,uuid,name,number
0,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,Cooperative Education Program,1
1,c070a84f-648f-351d-9499-5d0e30ad02cc,Cooperative Education/Co-op in Life Sciences C...,1
2,e6b4b7ae-0e0b-3aa5-9d77-7fcd90c9cfa3,Cooperative Education Program,1


In [6]:
course_offerings.limit(3).toPandas()

,uuid,course_uuid,term_code,name
0,344b3ebe-da7e-314c-83ed-9425269695fd,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1092,Cooperative Education Prog
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1082,Cooperative Education Prog
2,ea3b717c-d66b-30dc-8b37-964d9688295f,a3e3e1c3-543d-3bb5-ae65-5f2aec4ad1de,1172,Cooperative Education Prog


## Recap: About this database

You will notice that there are several more tables in the uw-madision-courses folder than there are read in above. This so that you will have a chance to practice your own custom joins and learn about the relationships between a real database work. Sometimes we don't know how they are related and we need to figure it out! I'll save that for the HW :) 

Here is a look at some of the important variables we can use to join our tables:

 - course_offerings: uuid, course_uuid, term_code, name
 - instructors: id, name
 - schedules: uuid
 - sections: uuid, course_offering_uuid,room_uuid, schedule_uuid
 - teachings: instructor_id, section_uuid
 - courses: uuid
 - grade_distributions: course_offering_uuid,section_number
 - rooms: uuid, facility_code, room_code
 - subjects: code
 - subject_memberships: subject_code, course_offering_uuid
 
 **Source:** https://www.kaggle.com/Madgrades/uw-madison-courses
 
So alright, let's use this information to discover some insights from this data!

## 1a. Can you assign the room numbers to each section of each course?

Show only the rooms uuid, facility code, room number, term code and the name of the course from the course_offerings table.

In [7]:
step1 = course_offerings.join(
    sections, course_offerings["uuid"] == sections["course_offering_uuid"], how="inner"
).select([course_offerings["uuid"], course_offerings["term_code"], course_offerings["name"], sections["room_uuid"]])

In [8]:
step1.toPandas()

,uuid,term_code,name,room_uuid
0,344b3ebe-da7e-314c-83ed-9425269695fd,1092,Cooperative Education Prog,null
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,1082,Cooperative Education Prog,null
2,ea3b717c-d66b-30dc-8b37-964d9688295f,1172,Cooperative Education Prog,04368a56-c959-3e4b-8b3d-f4cc3538fea5
3,ea3b717c-d66b-30dc-8b37-964d9688295f,1172,Cooperative Education Prog,null
4,ea3b717c-d66b-30dc-8b37-964d9688295f,1172,Cooperative Education Prog,null
...,...,...,...,...
315597,3573e335-e7be-33a6-b023-8704b3acc93b,1182,Conservation Planning,409b3445-80fc-3ca2-a44e-f254ba48d5cd
315598,5d437083-a1d0-38b8-aa70-ec3564da39a6,1182,null,null
315599,13278922-0726-3823-abda-dc14df03d8a6,1182,Patient Ed. as Advocacy Strat.,null
315600,fd039fe4-32ec-31dd-a142-c7db54df784b,1182,Multid. Appr. to Colon Cancer,null


In [9]:
step2 = step1.join(
    rooms, step1["room_uuid"] == rooms["uuid"], how="inner"
).select([step1["uuid"], step1["term_code"], step1["name"], rooms["uuid"], rooms["room_code"], rooms["facility_code"]])

In [10]:
step2.toPandas()

,uuid,term_code,name,uuid,room_code,facility_code
0,ea3b717c-d66b-30dc-8b37-964d9688295f,1172,Cooperative Education Prog,04368a56-c959-3e4b-8b3d-f4cc3538fea5,null,OFF CAMPUS
1,ea3b717c-d66b-30dc-8b37-964d9688295f,1172,Cooperative Education Prog,04368a56-c959-3e4b-8b3d-f4cc3538fea5,null,OFF CAMPUS
2,075da420-5f49-3dd0-93df-13e3c152e1b1,1114,Cooperative Education Prog,04368a56-c959-3e4b-8b3d-f4cc3538fea5,null,OFF CAMPUS
3,2b4e216d-a728-3713-8c7c-19afffc6b2fd,1104,Cooperative Education Prog,04368a56-c959-3e4b-8b3d-f4cc3538fea5,null,OFF CAMPUS
4,ff511882-5eab-3e7b-a89f-8fbfd1906127,1134,Cooperative Education Prog,04368a56-c959-3e4b-8b3d-f4cc3538fea5,null,OFF CAMPUS
...,...,...,...,...,...,...
209639,3573e335-e7be-33a6-b023-8704b3acc93b,1182,Conservation Planning,409b3445-80fc-3ca2-a44e-f254ba48d5cd,1170,0140
209640,3573e335-e7be-33a6-b023-8704b3acc93b,1182,Conservation Planning,33f69fe2-fe56-3b35-9302-7b6d1954b556,0175,0053
209641,3573e335-e7be-33a6-b023-8704b3acc93b,1182,Conservation Planning,33f69fe2-fe56-3b35-9302-7b6d1954b556,0175,0053
209642,3573e335-e7be-33a6-b023-8704b3acc93b,1182,Conservation Planning,33f69fe2-fe56-3b35-9302-7b6d1954b556,0175,0053


## 1b. Now show same output as above but for only facility number 0469 (facility_code)

In [11]:
step2.filter("facility_code = 0469").toPandas()

,uuid,term_code,name,uuid,room_code,facility_code
0,bd5baa02-e0ca-39df-a61d-361e8534f398,1152,Fundamentals-Flute,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,2441,0469
1,2101e24a-f58f-33d1-8ca1-61778a83f886,1092,Fundamentals-Flute,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,2441,0469
2,821927cc-c5eb-38c4-93f4-bcca6ee066b1,1172,Fundamentals-Flute,6af80b0b-b3e3-370a-925f-829869f007bc,4411,0469
3,a35e69d8-4355-3bf7-83f7-21f09212cb41,1162,Fundamentals-Flute,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,2441,0469
4,3e551eb9-a026-3ebb-98e9-3d4391c3bafd,1132,Fundamentals-Flute,9759cb5f-a7d3-3d0c-b8b3-03910bd0153e,2441,0469
...,...,...,...,...,...,...
16328,e72b604f-2517-3005-90a6-49ebba29f499,1182,Sex and Power,00593bf2-b562-3482-9bb3-e2b5e7e00a16,1101,0469
16329,3a480dd7-2187-3ba5-a16f-373d0420a60c,1182,Chaucers Courtly Poetry,0fabcc36-e3c1-32c3-99a9-c7c6ae53e992,1221,0469
16330,b5947db7-bec2-380a-bcd2-96483da237e1,1182,Graduate Choral Union,f4781736-d27f-3902-9095-5b856acb6db3,2340,0469
16331,b5947db7-bec2-380a-bcd2-96483da237e1,1182,Graduate Choral Union,f4781736-d27f-3902-9095-5b856acb6db3,2340,0469


## 2. Count how many sections are offered for each subject for each facility

*Note: this will involve a groupby*

In [12]:
step1 = (
    subjects.join(
        subject_memberships,
        subjects.code == subject_memberships.subject_code,
        how="inner",
    )
    .select(["name", "course_offering_uuid"])
    .withColumnRenamed("name", "subject_name")
)
step1.limit(4).toPandas()

,subject_name,course_offering_uuid
0,Chemical and Biological Engineering,344b3ebe-da7e-314c-83ed-9425269695fd
1,Electrical and Computer Engineering,344b3ebe-da7e-314c-83ed-9425269695fd
2,Engineering Mechanics and Astronautics,344b3ebe-da7e-314c-83ed-9425269695fd
3,Mechanical Engineering,344b3ebe-da7e-314c-83ed-9425269695fd


In [13]:
step2 = step1.join(
    sections, step1.course_offering_uuid == sections.course_offering_uuid, how="left"
).select(["subject_name", "room_uuid"])
step2.limit(4).toPandas()

,subject_name,room_uuid
0,Communication Arts,4948c250-c6cf-3272-a497-f90962f3ba67
1,Materials Science and Engineering,3767a755-bf46-3d70-b607-1ded3f01fb18
2,Chemical and Biological Engineering,null
3,Electrical and Computer Engineering,null


In [14]:
# I added a filter to make this a little simpler
step3 = (
    step2.join(rooms, step2.room_uuid == rooms.uuid, how="left")
    .filter('facility_code IN("0140","0545","0469","0031")')
    .select(["subject_name", "facility_code", "room_code"])
)
step3.limit(4).toPandas()

,subject_name,facility_code,room_code
0,Music,0469,1351
1,Music,0469,2340
2,Accounting and Information Systems,0140,1190
3,Accounting and Information Systems,0140,2170


In [15]:
# Option 1: Group by facility code and do a count
step3.groupBy("facility_code", "subject_name").count().orderBy("facility_code").show(
    10, False
)  # False prevents truncation of column content

+-------------+---------------------------------------+-----+
|facility_code|subject_name                           |count|
+-------------+---------------------------------------+-----+
|0031         |PHYSICAL EDUC ACTIVITY PROGM           |341  |
|0031         |Biochemistry                           |2    |
|0031         |Kinesiology                            |1549 |
|0031         |Interdisciplinary Courses (Engineering)|1    |
|0031         |Political Science                      |3    |
|0031         |History                                |1    |
|0031         |Spanish (Spanish and Portuguese)       |1    |
|0031         |East Asian Languages and Literature    |32   |
|0031         |History of Science                     |1    |
|0031         |Agricultural and Applied Economics     |3    |
+-------------+---------------------------------------+-----+
only showing top 10 rows



In [16]:
# Option 2: Groupby subject name and pivot the facility code
# to see each facility side by side within each subject
step3.groupBy("subject_name").pivot("facility_code").count().show(10, False)

+---------------------------------+----+----+----+----+
|subject_name                     |0031|0140|0469|0545|
+---------------------------------+----+----+----+----+
|Asian American Studies           |null|6   |245 |24  |
|Religious Studies                |null|32  |282 |43  |
|HEBREW                           |null|7   |4   |null|
|Botany                           |null|4   |24  |2   |
|Urban and Regional Planning      |null|208 |17  |6   |
|Nutritional Sciences             |6   |null|null|28  |
|Kinesiology                      |1549|null|null|null|
|Philosophy                       |1   |80  |1296|73  |
|Art Education (Department of Art)|null|null|145 |null|
|Interdisciplinary Courses (CALS) |null|5   |7   |28  |
+---------------------------------+----+----+----+----+
only showing top 10 rows



## 3. What are the hardest classes?

Let's see if we can figure out which classes are the hardest by seeing how many students failed. Note that you will first need to aggregate the grades table by the course uuid to include all sections. Show the name of the course as well that you will need to get from the course_offering table.

In [17]:
grade_distributions.limit(4).toPandas()

,course_offering_uuid,section_number,a_count,ab_count,b_count,bc_count,c_count,d_count,f_count,s_count,u_count,cr_count,n_count,p_count,i_count,nw_count,nr_count,other_count
0,344b3ebe-da7e-314c-83ed-9425269695fd,1,105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,f718e6cd-33f0-3c14-a9a6-834d9c3610a8,1,158,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,ea3b717c-d66b-30dc-8b37-964d9688295f,1,139,12,2,0,3,0,0,0,0,0,0,0,0,0,0,0
3,075da420-5f49-3dd0-93df-13e3c152e1b1,1,87,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [18]:
step1 = grade_distributions.groupBy("course_offering_uuid").sum("f_count")
step1.limit(4).toPandas()

,course_offering_uuid,sum(f_count)
0,c939dd5e-43ba-3266-8f74-573f629de75b,0
1,75fdf27b-0e52-3544-96eb-d594a5ad969f,0
2,cfeba94d-8e0e-320b-a904-ea2c2a31c000,7
3,ceabe145-78e5-33c2-9b68-3a1eea9c2864,4


In [20]:
step2 = (
    step1.join(
        course_offerings,
        step1.course_offering_uuid == course_offerings.uuid,
        how="left",
    )
    .select(["name", "sum(f_count)"])
    .orderBy("sum(f_count)")
)
step2.toPandas()

,name,sum(f_count)
0,Workshop in Dance Activity,0
1,Chorale,0
2,Ideas of Modern Physics,0
3,Chemistry in Our World,0
4,Academic Writing I,0
...,...,...
80166,Calculus&Analytic Geometry,63
80167,Calculus&Analytic Geometry 1,64
80168,Calculus&Analytic Geometry,67
80169,Animal Biology,70


## Challenge Question: Automating data entry errors

We see in the dataframe below that there are several typos of various animal names. If this was a large database of several millions of records, correcting these errors would be way too labor intensive. How can we automate correcting these errors?

*Hint: Leven...*

In [48]:
from pyspark.sql.functions import levenshtein, when
values = [
    ("Monkey", 10),
    ("Monkay", 36),
    ("Mnky", 123),
    ("Elephant", 48),
    ("Elefant", 16),
    ("Ellafant", 1),
    ("Hippopotamus", 48),
    ("Hipopotamus", 16),
    ("Hippo", 1),
]
zoo = spark.createDataFrame(values, ["Animal", "age"])
zoo.show()

+------------+---+
|      Animal|age|
+------------+---+
|      Monkey| 10|
|      Monkay| 36|
|        Mnky|123|
|    Elephant| 48|
|     Elefant| 16|
|    Ellafant|  1|
|Hippopotamus| 48|
| Hipopotamus| 16|
|       Hippo|  1|
+------------+---+



In [42]:
valid_animals = spark.createDataFrame([("Monkey",), ("Elephant",), ("Hippopotamus",)], ["Valid_Animals"])
valid_animals.show()

+-------------+
|Valid_Animals|
+-------------+
|       Monkey|
|     Elephant|
| Hippopotamus|
+-------------+



In [44]:
lev_dist = zoo.crossJoin(valid_animals).select("Animal", "Valid_Animals",
                                    levenshtein("Animal", "Valid_Animals").alias("Lev"))
lev_dist.toPandas()

,Animal,Valid_Animals,Lev
0,Monkey,Monkey,0
1,Monkey,Elephant,8
2,Monkey,Hippopotamus,11
3,Monkay,Monkey,1
4,Monkay,Elephant,7
5,Monkay,Hippopotamus,10
6,Mnky,Monkey,2
7,Mnky,Elephant,8
8,Mnky,Hippopotamus,12
9,Elephant,Monkey,8


In [53]:
lev_dist.withColumn("Modified", when(
    lev_dist["Lev"] < 5, lev_dist["Valid_Animals"]).otherwise(lev_dist["Animal"])).toPandas()

,Animal,Valid_Animals,Lev,Modified
0,Monkey,Monkey,0,Monkey
1,Monkey,Elephant,8,Monkey
2,Monkey,Hippopotamus,11,Monkey
3,Monkay,Monkey,1,Monkey
4,Monkay,Elephant,7,Monkay
5,Monkay,Hippopotamus,10,Monkay
6,Mnky,Monkey,2,Monkey
7,Mnky,Elephant,8,Mnky
8,Mnky,Hippopotamus,12,Mnky
9,Elephant,Monkey,8,Elephant


In [54]:
results = zoo.join(valid_animals, levenshtein(zoo["Animal"], valid_animals["Valid_Animals"]) < 5, "left")
results.show()

+------------+---+-------------+
|      Animal|age|Valid_Animals|
+------------+---+-------------+
|      Monkey| 10|       Monkey|
|      Monkay| 36|       Monkey|
|        Mnky|123|       Monkey|
|    Elephant| 48|     Elephant|
|     Elefant| 16|     Elephant|
|    Ellafant|  1|     Elephant|
|Hippopotamus| 48| Hippopotamus|
| Hipopotamus| 16| Hippopotamus|
|       Hippo|  1|         null|
+------------+---+-------------+



### Great job! 